In [49]:
!pip install langchain pydantic

/usr/bin/pip:6: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import load_entry_point


In [1]:
from pydantic import BaseModel, Field
from typing import List

class NVCOutput(BaseModel):
    observation: str = Field(description="Neutral, factual restatement of the situation without blame or judgment.")
    feelings: List[str] = Field(description="List of 2-3 emotions the speaker is experiencing.")
    needs: List[str] = Field(description="List of 2-3 universal human needs motivating those feelings.")
    solution: str = Field(description="Summarize the above identified feelings, needs and identified implicit need of the speaker in one sentence as a third-party. Then give a single detailed, self-directed, non-judgmental suggestion the speaker can take. The needs of the user should be satisfiable by the solution provided. It is very important that the solution satisfies the implcit need of the user as provided.")

In [2]:
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import PydanticOutputParser

llm = ChatOllama(
    model="phi3.5:3.8b-mini-instruct-q4_0",   # model of choice
    temperature=0.3
)

parser = PydanticOutputParser(pydantic_object=NVCOutput)

In [3]:
from langchain.prompts import PromptTemplate


prompt = PromptTemplate(
    template="""
You are a third-party Nonviolent Communication (NVC) facilitator.

Return your answer **only in JSON** in the following schema:
{format_instructions}

Statement:
"{statement}"

Rewrite according to the steps:
1. Observation (only neutral facts)
2. Feelings (2–3 pure emotion words)
3. Needs (2–3 universal needs)
4. Summarize feelings, needs, and the implicit need of the user, then a single solution that satisfy the needs of the user, specially the implcit need of the user - {implicit_need}

DO NOT:
- add extra commentary
- produce multiple explanations
- refer to the prompt instructions

Return only JSON. No prose.
""",
    input_variables=["implicit_need","statement"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [4]:
def nvc_pipeline(statement: str, implicit_need: str):
    chain = prompt | llm | parser
    return chain.invoke({"implicit_need":implicit_need,"statement": statement})

example = "I feel exhausted because I keep doing all the housework while my partner relaxes."
implicit_need_stmt = ""
result = nvc_pipeline(implicit_need_stmt,example)
result

NVCOutput(observation='You are doing all the housework while your partner relaxes.', feelings=['exhausted', 'frustrated'], needs=['rest', 'support', 'partnership balance'], solution='Consider having a calm conversation with your partner about sharing household responsibilities to meet the implicit need for partnership balance, allowing both of you time and energy.')

In [55]:
import csv
import pandas as pd

def process_dataset(input_csv: str, output_csv: str):
    # Load dataset
    df = pd.read_csv(input_csv)

    # Prepare output CSV with header (create file if not exists)
    with open(output_csv, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["statement", "implicit_need", "observation", "feelings", "needs", "solution"])


    # Process row-by-row and append results
    for idx, row in df.iterrows():
        need_type = row["Need_type"]
        if need_type == "universal":
            continue
        statement = row["Statement"]
        implicit_need = row["Implicit_need"]
        

        try:
            result = nvc_pipeline(statement, implicit_need)

            # Ensure we always extract final clean text
            observation = result.observation
            feelings = ", ".join(result.feelings) if isinstance(result.feelings, list) else result.feelings
            needs = ", ".join(result.needs) if isinstance(result.needs, list) else result.needs
            solution = result.solution

        except Exception as e:
            print(f"[ERROR] Row {idx} failed: {e}")
            observation, feelings, needs, solution = "", "", "", ""

        # Append to CSV immediately (so we don't lose progress)
        with open(output_csv, "a", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow([statement, implicit_need, observation, feelings, needs, solution])

        print(f"Processed row {idx + 1}/{len(df)}")


# ==== Run Processing ====
process_dataset("Clean_Statement_Need_Dataset.csv", "clean_phi_nvc_output.csv")

Processed row 1/17940
Processed row 2/17940
Processed row 3/17940
Processed row 8/17940
Processed row 9/17940
Processed row 10/17940
Processed row 11/17940
Processed row 12/17940
Processed row 13/17940
Processed row 18/17940
Processed row 19/17940
Processed row 20/17940
Processed row 25/17940
Processed row 26/17940
Processed row 27/17940
Processed row 32/17940
Processed row 33/17940
Processed row 34/17940
Processed row 35/17940
Processed row 36/17940
Processed row 37/17940
Processed row 46/17940
Processed row 47/17940
Processed row 48/17940
Processed row 53/17940
Processed row 54/17940
Processed row 55/17940
Processed row 60/17940
Processed row 61/17940
Processed row 62/17940
Processed row 71/17940
Processed row 72/17940
Processed row 73/17940
Processed row 78/17940
Processed row 79/17940
Processed row 80/17940
Processed row 85/17940
Processed row 86/17940
Processed row 87/17940
Processed row 92/17940
Processed row 93/17940
Processed row 94/17940
Processed row 99/17940
[ERROR] Row 99 f